# Topic 5: Multilayer Perceptron
## Associated Reading: Bishop 5.1, 5.2, 5.3

# 1 When does logistic regression fail?
Imagine that you're a wildlife biologist, and you're conducting a study on the activity habits of whitetail deer.  
<img src="images/deer.jpg">
As any hunter knows, deer activity varies depending on time of day, and this time-varying level of activity is what you're tasked with quantifying.  In particular, what you want to do is to fit a model that asks the question: if I am sitting at a particular spot at a particular time of day, what is the probability that I will see a whitetail deer?  

In principle, this is precisely the type of question that we might be interested in using logistic regression for.  We have a feature $\mathbf{x}$, which is the time of day, as well as a binary outcome $y\in\{0,1\}$ (whether we see a deer or not) for which we'd like to generate a probability.  What do we need to fit this model?  

Of course, to begin with we'll need a dataset.  To collect such a dataset we could simply put out a camera (strapped to a tree, for example), set it to take pictures at random and determine whether there is a deer in the image (we could determine this by hand or using a machine learning algorithm to automatically process the image, something that we'll get to in this course!).  In full disclosure, the dataset that I'll provide here is synthetically generated (i.e. it's from a simulation).  However, let's suspend disbelief and imagine that we've done what we described above. 

In [ ]:
import numpy as np
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [18,15]
mpl.rcParams['font.size'] = 18
import matplotlib.pyplot as plt

x,y = np.loadtxt('datasets/deer.npy')
x = x[::5]
y = y[::5]
idx = np.argsort(x)
x = x[idx]
y = y[idx]
y_obs = np.reshape(y,(len(y),1))



plt.hist(x[y==0],bins=24,histtype='step',label='no deer')
plt.hist(x[y==1],bins=24,histtype='step',label='deer')
plt.xlabel('Normalized time')
plt.ylabel('Count')
plt.legend()
plt.show()

Now we'd like to fit a logistic regression model.  As with the lobster problem, it's helpful to have normalized features so that gradient descent converges easily.  We can basically lift the code from that problem for running gradient descent:

In [ ]:
def sigmoid(a):
    return 1./(1+np.exp(-a))

def L(y_obs,y_pred):
    return -1./len(y_obs)*np.sum(y_obs*np.log(y_pred) + (1-y_obs)*np.log(1-y_pred))

def grad(y_obs,y_pred,phi):
    return phi.T @ (y_pred-y_obs)

w = np.random.randn(2,1)
phi = np.vstack((np.ones_like(x),x)).T

eta = 1e-4
for i in range(50):
    y_pred = sigmoid(phi @ w)
    w -= eta*grad(y_obs,y_pred,phi)  
    print(L(y_obs,y_pred))

In [ ]:
y_obs.shape

This is a pretty easy problem, so gradient descent converges in just a handful of iterations.  Let's plot our predictions.  We'll just plot the histograms from before, as well as the outut of the sigmoid. 

In [ ]:
plt.hist(x[y==0],bins=24,histtype='step',label='no deer')
plt.hist(x[y==1],bins=24,histtype='step',label='deer')
plt.xlabel('Normalized time')
plt.ylabel('Count')
plt.legend()
ax = plt.twinx()
ax.plot(x,y_pred,'k.')
ax.set_ylabel('Probability of deer sighting')
ax.set_ylim(0,1)
plt.show()

Huh.

It appears to be the case that logistic regression is telling us that there always about a 40% chance of seeing a deer, regardless of the time of day.  While this is a bit underwhelming, logistic regression is actually performing as well as can be expected in this case: gradient descent has been successful, and this is indeed the optimized model.  Please answer the following two questions:
- **Why is the model predicting a uniform value for the probability, even though the data clearly show probability as a function of time?**
- **Why does the model predict 40% everywhere?**

We can perhaps gain some insight here by attempting to adjust the sigmoid by hand.  Let's start with some random one.

In [ ]:
plt.hist(x[y==1],bins=24,histtype='step',label='deer')
plt.xlabel('Normalized time')
plt.ylabel('Count')
plt.legend()

ax = plt.twinx()
w_random = np.array([-5,10])
y_pred = sigmoid(phi@w_random)
ax.plot(x,y_pred,'k.')
ax.set_ylabel('Probability of deer sighting')
ax.set_ylim(0,1)
plt.show()

We want this curve to be high whenever the sighting count is high and low otherwise.  We have two modes of adjustment.  We can vary $w_0$, which does this:

In [ ]:
plt.hist(x[y==1],bins=24,histtype='step',label='deer')
plt.xlabel('Normalized time')
plt.ylabel('Count')
plt.legend()

ax = plt.twinx()
for w_0 in range(-8,-3):
    w_random = np.array([w_0,10])
    y_pred = sigmoid(phi@w_random)
    ax.plot(x,y_pred,'.')
ax.set_ylabel('Probability of deer sighting')
ax.set_ylim(0,1)
plt.show()

Or we can adjust $w_1$, which does this:

In [ ]:
plt.hist(x[y==1],bins=24,histtype='step',label='deer')
plt.xlabel('Normalized time')
plt.ylabel('Count')
plt.legend()

ax = plt.twinx()
for w_1 in range(5,15,2):
    w_random = np.array([-5,w_1])
    y_pred = sigmoid(phi@w_random)
    ax.plot(x,y_pred,'.')
ax.set_ylabel('Probability of deer sighting')
ax.set_ylim(0,1)
plt.show()

**Is there a good way to adjust the logistic function using these two knobs such that it gives a good prediction everywhere?**  The answer, of course, is no: the bimodality of the data makes it so that it's quite impossible for the logistic function, which can only split the domain into one region each of high probability and low probability, to fit well.  As such, the optimization procedure essentially throws up its hands and says "the best I can do is to default to the prior distribution, which is about a 40% chance of success.  This is pretty much what will *always* happen with logistic regression whenever there isn't a single point that represents a sensible decision boundary between the success and failure class (this boundary becomes a line in 2D, a plane in 3D, and so on).

## 5.2 Data transformation
The problem isn't necessarily hopeless though.  Notice that the features are fixed and known.  As such, we could try to transform the features into something that is more amenable to the assumptions of logistic regression.  To put this concretely, rather than using $x$ (the time) as a feature, we could use $f(x)$ as a feature, where $f(x)$ is some function that we can choose.  For example, we could hypothesize that maybe taking the negative exponential of the feature might yield a better behaved problem: 

In [ ]:
xhat = np.exp(-x)
plt.hist(xhat[y==0],bins=24,histtype='step',label='no deer')
plt.hist(xhat[y==1],bins=24,histtype='step',label='deer')
plt.xlabel('f(Normalized time)')
plt.ylabel('Count')
plt.legend()
plt.show()

But of course it doesn't, because $f(x) = \mathrm{e}^{-x}$ is monotonic.  **Come up with some function that would transform the data such that the notion of a single, pointwise decision boundary becomes more reasonable.** 

In [ ]:
def f(x):
    """ Define a function that transforms the data into something that can be used with logistic regression."""
    return np.ones_like(x)  # This is just a placeholder

xhat = f(x)
plt.hist(xhat[y==0],bins=24,histtype='step',label='no deer')
plt.hist(xhat[y==1],bins=24,histtype='step',label='deer')
plt.show()

<img src="images/fighting.jpg">

Once you've completed the above exercise, you can look at the very reasonable choice that I've suggested below.

In [ ]:
xhat = np.cos(4*np.pi*x)
plt.hist(xhat[y==0],bins=24,histtype='step',label='no deer')
plt.hist(xhat[y==1],bins=24,histtype='step',label='deer')
plt.show()


By taking the cosine (adjusted for twice-daily surge of deer activity), we transform the data into two unimodal populations, which is just what we'd like for using logistic regression.  In fact, we can fit this dataset using logistic regression pretty well.

In [ ]:
w = np.random.randn(2,1)
phi = np.vstack((np.ones_like(xhat),xhat)).T

eta = 1e-3
for i in range(50):
    y_pred = sigmoid(phi @ w)
    w -= eta*grad(y_obs,y_pred,phi)  
    print(L(y_obs,y_pred))

In [ ]:
plt.hist(xhat[y==0],bins=24,histtype='step',label='no deer')
plt.hist(xhat[y==1],bins=24,histtype='step',label='deer')
plt.xlabel('cos(4 pi x)')
plt.ylabel('Count')
plt.legend()
ax = plt.twinx()
ax.plot(xhat,y_pred,'k.')
ax.set_ylabel('Probability of deer sighting')
ax.set_ylim(0,1)
plt.show()

In $f(x)=\mathrm{cos \;4\pi x}$-space, we get a pretty nice fit to the data.  However, it's easier to interpret back in normal $x$ space.  Since the transformation between $x$ and $f(x)$ is fixed and known, it's easy to plot the probabilities as a function of $x$, rather than as a function of $f(x)$.

In [ ]:
plt.hist(x[y==0],bins=24,histtype='step',label='no deer')
plt.hist(x[y==1],bins=24,histtype='step',label='deer')
plt.xlabel('Normalized time')
plt.ylabel('Count')
plt.legend()
ax = plt.twinx()
ax.plot(x,y_pred,'k.')
ax.set_ylabel('Probability of deer sighting')
ax.set_ylim(0,1)
plt.show()

Pretty nice.  We've established that even though logistic regression doesn't work very well for certain patterns of data (i.e. bimodal data), it can work very well if we can transform the features such that the assumptions of logistic regression are satisfied.

## 5.3 The trouble
While data transformation is very powerful, using it as we did above becomes problematic in more challenging cases.  In particular, our capacity to use expert judgement to come up with a sensible function to use as a transformation mechanism becomes prohibitive as the dimensionality of the data becomes large.  Thus, we need to come up with an *automatic* method for transforming this data such that it is amenable for use in logistic (or softmax) regression.

## 5.4 Graphical models
Before we tackle the problem of automatic data transformation, it will be useful to write our models graphically, just to keep things straight.  We'll write square nodes for input features, and lines indicating multiplication, with the coefficient of that multiplication written above the line.  Circular nodes will represent a function that takes as input $a$, which is a sum of all the inputs going to that node, and outputs $z$.  With this notation we can easily write normal logistic regression as
<img src="addme.jpg">
The model that we came up with above would be written as
<img src="addme.jpg">
If we're interested in outputting an objective function value from our graph instead of a prediction, that just adds another node onto the end of the graph.  

## 5.5 Hidden layers
Each of these models are in fact (very simple) neural networks, where we map directly from input to prediction to cost function, by the way of these simple activation functions: the identity corresponds to values on the real line, the sigmoid to predicting binary classses, etc.  These activations act on fixed basis functions specified by the input: the ones column, the $x_1$ column, etc.  What if we decided to incorporate an intermediate layer that could somehow mix up the basis functions into alternative basis functions that we would then feed to our activation.  For example, what if instead of just feeding weighted combinations of features to the logistic function, we instead preprocessed those data into a different set of basis functions?  Perhaps we wouldn't be able to understand exactly what those basis functions were, but they would give our model more flexibility.  

Let's introduce another set of activation functions that our inputs pass through before they reach the sigmoid.  This layer of nodes is referred to as a \emph{hidden layer}.  In principle, we could have as many as we want.  These activation functions \emph{must} be non-linear for this to be useful.  **What happens when we use the identity as an activation function in this intermediate layer?:**  

In practice, using the logistic function itself is a popular choice, but there are many(!) other possibilities.  The primary use of this is that a linear combination of sigmoids is a *universal function approximator*, which is to say that if we take the linear combination of enough of these things, then we can approximate any function that we want to.  For example, what if we wanted to approximate a bell curve: we could use two sigmoids with opposite signs and different translations, which would add up to something approximating a bell curve.  What happens if we wanted to approximate a line?  This wouldn't work because there are these plateaus, right?  It still works, because we could just make one of our sigmoids highly diffuse, so that only this roughly linear interior section is in play, then simply ignore the second one by setting its output weight to zero.  Two sigmoids actually have the capacity to roughly mimic pretty much anything that is linear of unimodal:  pretty much anything with one peak or less.  If we increase the number of nodes in the hidden layer, then we can mimic even more functions, like a sinusoid: for our deer problem, we can use a bank of sigmoids to approximate the sinusoidal transformation we came up with heuristically above!  In particular, we'll use the following graph
<img src="addme.jpg">

Note that this structure is often called a multi-layer perceptron.  This is sort of an unfortunate name, because it does not, in fact use perceptrons anywhere.  A perceptron is just a funny name for Heaviside regression, which is just like logistic regression, but instead of using the regression function, it uses the heaviside function.  It turned out to not be a particularly useful method for classification, but for some reason the name stuck when it came to neural networks.    

## 5.6 Feedforward neural network
The complexity of this graph belies the fact that it's actually pretty simple to write a mathematical expression here.  The computation proceeds sequentially
$$
A^{(1)} = X W^{(1)} + b^{(1)}
$$
$$
Z^{(1)} = \sigma(A^{(1)})
$$
$$
A^{(2)} = Z^{(1)} W^{(2)} + b^{(2)}
$$
$$
Z^{(2)} = y_{pred} = \sigma(A^{(2)}).
$$
**work out the dimensions of this computation, and ensure that they make sense**.  Running a set of features $X$ through this set of linear and non-linear functional compositions leads to a binary prediction, just as in logistic regression.  But how do we train this model?  We now have two matrices of parameters $W$ (of different sizes).  How do we proceed?



In [ ]:
m = len(x)
n_0 = 1
n_1 = 4
N = 1

np.random.seed(42)

X = x.reshape((m,n_0))

W1 = 10*np.random.randn(n_0,n_1)
W2 = 10*np.random.randn(n_1,N)

b1 = 10*np.random.randn(1,n_1)#np.array([0.,4,8,12.])
b2 = 10*np.random.randn(1,N)


def sigmoid(a):
    return 1./(1+np.exp(-a))

def L(y_obs,y_pred):
    return -np.sum(y_obs*np.log(y_pred) + (1-y_obs)*np.log(1-y_pred))

def grad(y_obs,y_pred,phi):
    return phi.T @ (y_pred-y_obs)


def feedforward(X,W1,W2,b1,b2):
    # Feedforward
    A1 = X@W1 + b1
    Z1 = sigmoid(A1)
    A2 = Z1@W2 + b2
    y_pred = sigmoid(A2)
    return y_pred,Z1

y_pred,Z1 = feedforward(X,W1,W2,b1,b2)
print(y_pred)
plt.plot(X,y_pred,'k.')

## 5.7 Backpropagation

Despite the mystique, the neural network is just a model like any other (albeit an extraordinarily flexible one), written as
\begin{equation}
y_{pred} = F(X,W).
\end{equation}
For it to be useful, we still need to find parameter values: we need to train the model.  Consider the case of logistic regression, but with a hidden layer added.  This thing is really not so different from just vanilla logistic regression: we've added an additional layer of complexity, but the output remains the same, and as such, it's reasonable to assume the same misfit, and also to assume that we can solve this problem via the same technique: gradient descent. 

We seek the derivative of a cost function with respect to an arbitrary weight in the network
\begin{equation}
\frac{\partial J}{\partial W^{(l)}_{ij}}.
\end{equation}
As we saw, on graphs like the neural network, we can use the chain rule to propagate changes in misfit back through the network's function transformations, hence the name \emph{backpropagation}.  For the final layer, we proceed very similarly to how we did for the single layer networks:
\begin{equation}
\frac{\partial J}{\partial w^{(L)}_{ij}} = \frac{\partial J}{\partial z^{(L)}_{j}} \frac{\partial z^{(L)}_{j}}{\partial a_j^{(L)}} \frac{\partial a_j^{(L)}}{\partial w^{(L)}_{ij}}.
\end{equation}
Using our result from above, we have that
\begin{equation}
\frac{\partial J}{\partial w^{(L)}_{ij}} = \underbrace{(z^{(L)}_j - y)}_{\delta_j^{(L)}} z_{i}^{(L-1)},
\end{equation}
where we've specified the value $\delta_j^{(l)}$, which is the magnitude of the error signal propagated to $w_{ij}$.  With that in mind, we have 
\begin{equation}
\frac{\partial J}{\partial w^{(L)}_{ij}} = \underbrace{\delta_j^{(L)}}_{\text{backprop. error}} \underbrace{z_{i}^{(L-1)}}_{\text{forward prop. magnitude}}.
\end{equation}

For deeper network layers, the situation is marginally more complex.  For a weight $w_{ij}^{(l)}$, $l\neq L$, we perform the same chain rule differentiation:
\begin{equation}
\frac{\partial J}{\partial w^{(l)}_{ij}} = \frac{\partial J}{\partial z^{(l)}_{j}} \frac{\partial z^{(l)}_{j}}{\partial a_j^{l)}} \frac{\partial a_j^{(l)}}{\partial w^{(l)}_{ij}}.
\end{equation}
However, we can't directly compute $\frac{\partial J}{\partial z^{(l)}_{j}}$ anymore, because there are now other nodes in the way.  This isn't a problem however.  We can use the chain rule again, this time to expand  $\frac{\partial J}{\partial z^{(l)}_{j}}$:
\begin{equation}
\frac{\partial J}{\partial z^{(l)}_{j}} = \sum_{k=1}^{N^{(l+1)}} \frac{\partial J}{\partial z_k^{(l+1)}}\frac{\partial z_k^{(l+1)}}{\partial a_k^{(l+1)}} \frac{\partial a_k^{(l+1)}}{\partial z_j^{(l)}}.
\end{equation}  
This seems quite messy until you recognize that we \emph{already computed} the first two terms.  We even gave them a name: $\delta$.  This now simplifies to 
\begin{equation}
\frac{\partial J}{\partial z^{(l)}_{j}} = \sum_{k=1}^{N^{(l+1)}} \delta_k^{(l+1)} w^{(l+1)}_{jk},
\end{equation}
where we've also substituted $\frac{\partial a_k^{(l+1)}}{\partial z_j^{(l)}} = w^{(l+1)}_{jk}$.  We can now define a $\delta$ for the current layer
\begin{equation}
\delta_j^{(l)} = (\sum_{k=1}^{N^{(l+1)}} \delta_k^{(l+1)} w^{(l+1)}_{jk}) f_j'(a_j^{(l)}),
\end{equation}
where $f_j^{(l)}(\cdot)$ is the activation function for layer $l$, node $j$, and $f'(a_j^{(l)})$ is its derivative (with respect to $a_j^{(l)}$), which leads to
\begin{equation}
\frac{\partial J}{\partial w^{(l)}_{ij}} = \underbrace{\delta_j^{(l)}}_{\text{backprop. error}} \underbrace{z_{i}^{(l-1)}}_{\text{forward prop. magnitude}},
\end{equation}
which is the same as for layer $L$, just with a different definition for $\delta$.  It's also even more clear why this is called backpropagation: the gradient for a given layer depends upon the layer in front of it, so the error feeds backwards through the neural network, just as the input feeds forward through the neural network.  There exists this kind of beautiful duality to this structure, where the forward model sweeps from left to right, and the backwards model (sometimes called the adjoint) sweeps from right to left. 

For implementation on a computer, all those sums are unwieldy and don't allow us to use the fast matrix multiplication libraries that are available.  The tensor version of these equations for softmax with cross-entropy objective function is as follows.  Beginning with the feed forward stage:
\begin{equation}
A^{(l)} = Z^{(l-1)} W^{(l)} + B^{(l)}
\end{equation}
\begin{equation}
Z^{(l)} = \sigma(A^{(l)})
\end{equation}
Note that $Z^{(0)}=X$, where $X$ is the $m \times n$ feature matrix.  

For backpropagation, the tensor form is 
\begin{equation}
\nabla_{W^{(l)}} \mathcal{J} = (Z^{(l-1)})^T\delta^{(l)}
\end{equation}
\begin{equation}
\nabla_{B^{(l)}} \mathcal{J} = \mathbf{1}^T \delta^{(l)}
\end{equation}
where $\mathbf{1}$ is the $m \times 1$ vector of ones, and
\begin{equation}
\delta^{(l)} = \begin{cases} (Z^{(l)} - \mathcal{T}),\; \text{if }l=L \\
                              \delta^{(l+1)} (W^{(l+1)})^T \circ \sigma'^{(l)}(A^{(l)}),\;\text{else}, \end{cases} 
\end{equation}
where $\mathcal{T}$ is the one-hot representation of the data labels, and $\circ$ represents the Hadamard product (aka elementwise multiplication).  Interestingly, this is also valid for the case of sum square error coupled with an identity activation on the final node, i.e. for regression problems.  In some sense, the gradient of the cost function becomes simple when natural choices of output activation and cost function are chosen.

The following function implements the backpropagation formula given above.

In [ ]:
def backpropogate(y_pred,Z1,X,y_obs):
    # Backpropogate
    delta_2 = y_pred - y_obs
    grad_W2 = Z1.T @ delta_2
    grad_b2 = delta_2.sum(axis=0)

    delta_1 = delta_2 @ W2.T * Z1*(1-Z1)
    grad_W1 = X.T @ delta_1
    grad_b1 = delta_1.sum(axis=0) 
    return grad_W1,grad_W2,grad_b1,grad_b2


Note that we have gradients for *several* items: both weight matrices, as well as the bias vectors.  We can just update these separately, but all at the same time

In [ ]:
eta = 1e-3
for i in range(100000):
    if i>20000:
        eta = 1e-4
    y_pred,Z1 = feedforward(X,W1,W2,b1,b2)
    grad_W1,grad_W2,grad_b1,grad_b2 = backpropogate(y_pred,Z1,X,y_obs)

    W1 -= eta*np.sign(grad_W1)
    W2 -= eta*np.sign(grad_W2)
    b1 -= eta*np.sign(grad_b1)
    b2 -= eta*np.sign(grad_b2)
    if i%100==0:
        print(i,L(y_obs,y_pred))

After fitting, how do we do?  We can compare the output of our neural network against the success probability for an empirical histogram of training data (note that this is actually a form of nearest neighbor classification).

In [ ]:
c1,b = np.histogram(x[y==1],bins=np.linspace(0,1,48))
c2,b = np.histogram(x[y==0],bins=np.linspace(0,1,48))
P1 = c1/(c1+c2)
print(P1)
plt.plot(0.5*(b[1:]+b[:-1]),P1)
plt.plot(x,y_pred,'k.')

Pretty good results for a problem that would have thwarted one of our earlier classifiers.  Let's examine what this thing is doing a little bit more deeply.  It's particularly interesting to look at the outputs of the hidden layer, or what basis functions the model decided to transform the data to before classification. 

In [ ]:
for i in range(4):
    plt.plot(x[::100],Z1[::100,i],'k-')
    plt.scatter(x[::100],Z1[::100,i],c=y_obs[::100].ravel())
plt.show()

These basis functions represent a transform of our data to a new four dimensional space.  It's instructive to see what we get when we add them up and scale them by some weights that we found with gradient descent: the linear combination of these learned features that get passed to logistic regression:

In [ ]:
plt.scatter(Z1@W2 + b2,np.zeros((m)),c=y_obs)

Our neural network has effectively transformed our dataset into one that can reasonably be classified by the logistic function, just like we did manually via the cosine transform above!

## 5.8 A non-trivial example
We can also apply multilayer perceptrons to image data, just as we did with softmax.  This ends up being a pretty interesting exercise, because we can reshape the various weights on the first layer in order to give us a sense of **what features the neural networks is looking for**.  As it turns out, it does *not* look for the same combinations of features that softmax did.  

In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# In order to run this in class, we're going to reduce the dataset by a factor of 5
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X = X[::5]
y = y.astype(int)[::5]
X,X_test,y,y_test = train_test_split(X,y)

# Here we specify the size of our neural network.  Out hidden layer has n_1=256 nodes, 
# while our output (softmax) has 10 classes

m = len(X)
n_0 = 784
n_1 = 256
N = 10

# Function to convert categorical labels into one-hot matrix.
def convert_to_one_hot(y,n_classes):
    T = np.zeros((y.shape[0],n_classes))
    for t,yy in zip(T,y):
        t[yy]=1   
    return T
T = convert_to_one_hot(y,N) 
T_test = convert_to_one_hot(y_test,N)

After importing our data, now we just need to setup our neural network.

In [ ]:
np.random.seed(42)

# Initialize random parameter matrices
W1 = 0.001*np.random.randn(n_0,n_1)
W2 = 0.001*np.random.randn(n_1,N)

b1 = 0.1*np.random.randn(1,n_1)
b2 = 0.1*np.random.randn(1,N)

# Sigmoid function (activation)
def sigmoid(a):
    return 1./(1+np.exp(-a))

# Softmax function (final layer for classification)
def softmax(A):
    numerator = np.exp(A)
    denominator = numerator.sum(axis=1)
    return numerator/denominator[:,np.newaxis]  

# Categorical cross-entropy (what do these additional terms do?)
def L(T,S,W1,W2,alpha_1=1e-2,alpha_2=1e-5):
    return -1./len(T)*np.sum(T*np.log(S)) + np.sum(0.5*alpha_1*W1**2) + np.sum(0.5*alpha_2*W2**2)

# Run the neural network forward, given some weights and biases
def feedforward(X,W1,W2,b1,b2):
    # Feedforward
    A1 = X@W1 + b1
    Z1 = sigmoid(A1)
    A2 = Z1@W2 + b2
    y_pred = softmax(A2)
    return y_pred,Z1

# Compute the neural network gradients using backpropagation
def backpropogate(y_pred,Z1,X,y_obs,alpha_1=1e-2,alpha_2=1e-5):
    # Backpropogate
    delta_2 = (1./len(y_pred))*(y_pred - y_obs)
    grad_W2 = Z1.T @ delta_2 + alpha_2*W2
    grad_b2 = delta_2.sum(axis=0)

    delta_1 = delta_2 @ W2.T * Z1*(1-Z1)
    grad_W1 = X.T @ delta_1 + alpha_1*W1
    grad_b1 = delta_1.sum(axis=0) 
    return grad_W1,grad_W2,grad_b1,grad_b2

# Establish learning rate
eta = 1e-1

# Perform gradient descent
for i in range(500):
    y_pred,Z1 = feedforward(X,W1,W2,b1,b2)
    grad_W1,grad_W2,grad_b1,grad_b2 = backpropogate(y_pred,Z1,X,T)

    W1 -= eta*grad_W1
    W2 -= eta*grad_W2
    b1 -= eta*grad_b1
    b2 -= eta*grad_b2
    
    # Print some summary statistics every ten iterations
    if i%10==0:
        y_pred_test,Z1_test = feedforward(X_test,W1,W2,b1,b2)
        acc = sum(y_test==np.argmax(y_pred_test,axis=1))/len(y_test)
        print(i,L(T,y_pred,W1,W2),acc)

The input to each node in the hidden layer is a bunch of weights multiplied by a bunch of pixels and summed.  However, this time, rather than the resulting sums being interpreted as logits and going straight into the softmax, these sums become a new and transformed set of features, which then get multiplied with yet another set of weights before being input to softmax.  We can visualize this first set of weights as images, just as we did in the homework.

In [ ]:
import matplotlib.pyplot as plt
fig,axs = plt.subplots(nrows=5,ncols=5)
axx = axs.ravel()
for _,ax in enumerate(axx):
    ax.imshow(W1[:,np.random.randint(n_1)].reshape((28,28)))

These weights are certainly less interpretable!  Rather than making a fuzzy looking zero and asking whether our input looks like that, now the network is saying something more like "you see that group of pixels in the upper right hand corner? those are helpful pixels for making a prediction, so we'll see if the input image has any intensity there.  And then we'll look for that funny blob of pixels in the middle.  If the input image has the first but not the second, then it's probably a zero.  If it's got both, it's probably an eight".  And so on.  Speaking of which, it's interesting to look at a confusion matrix for this problem.

In [ ]:
y_pred_test = np.argmax(y_pred_test,axis=1)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred_test,y_test)

Not much to be surprised about here: zeros are often confused for fives and sixes, while fours are confused for nines.  

## IC5A Confused images
*Human* error rates on this dataset are typically about 3%.  Our neural network has about a 6% error rate, so we're not quite there.  I'd like you to find examples from the test set where the neural network classified *incorrectly*, and see if you can determine the difference (without looking at the labels).  Is there a pattern to which digits are incorrectly classified?  

## IC5B Extra layers
The network below implements a multilayer perceptron for the iris dataset that has a single hidden layer between the input and the softmax regression layer, which has 10 nodes.  **Add a second hidden layer after the first, which should have 15 nodes.**  Determine whether this added model complexity is justified by comparing the test set accuracy between the one-hidden-layer and two-hidden-layer models.

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

#Load the dataset
iris = datasets.load_iris()
X = iris.data 
y = iris.target
X,X_test,y,y_test = train_test_split(X,y)

m = len(x)  # Number of irises
n_0 = 4     # Number of input features
n_1 = 10    # Number of hidden layer features
N = 3       # Number of classes

# Function to convert categorical labels into one-hot matrix.
def convert_to_one_hot(y,n_classes):
    T = np.zeros((y.shape[0],n_classes))
    for t,yy in zip(T,y):
        t[yy]=1   
    return T
T = convert_to_one_hot(y,N) 
T_test = convert_to_one_hot(y_test,N)

# Instantiate random parameters to begin with
W1 = 0.0001*np.random.randn(n_0,n_1)
W2 = 0.0001*np.random.randn(n_1,N)

b1 = 0.0*np.random.randn(1,n_1)
b2 = 0.0*np.random.randn(1,N)

# sigmoid activation function
def sigmoid(a):
    return 1./(1+np.exp(-a))

# softmax classification function
def softmax(A):
    numerator = np.exp(A)
    denominator = numerator.sum(axis=1)
    return numerator/denominator[:,np.newaxis]  

# Categorical cross-entropy
def L(T,S,W1,W2):
    return -1./len(T)*np.sum(T*np.log(S))

# Run the neural network forward
def feedforward(X,W1,W2,b1,b2):
    # Feedforward
    A1 = X@W1 + b1
    Z1 = sigmoid(A1)
    A2 = Z1@W2 + b2
    y_pred = softmax(A2)
    return y_pred,Z1

# Compute gradients with backpropagation
def backpropogate(y_pred,Z1,X,y_obs):
    # Backpropogate
    delta_2 = (1./len(y_pred))*(y_pred - y_obs)
    grad_W2 = Z1.T @ delta_2
    grad_b2 = delta_2.sum(axis=0)

    delta_1 = delta_2 @ W2.T * Z1*(1-Z1)
    grad_W1 = X.T @ delta_1 
    grad_b1 = delta_1.sum(axis=0) 
    return grad_W1,grad_W2,grad_b1,grad_b2

# set learning rate
eta = 1

# learn for a set number of epochs
for i in range(1000):
    y_pred,Z1 = feedforward(X,W1,W2,b1,b2)
    grad_W1,grad_W2,grad_b1,grad_b2 = backpropogate(y_pred,Z1,X,T)

    W1 -= eta*grad_W1
    W2 -= eta*grad_W2
    b1 -= eta*grad_b1
    b2 -= eta*grad_b2
    if i%10==0:
        y_pred_test,Z1_test = feedforward(X_test,W1,W2,b1,b2)
        acc = sum(y_test==np.argmax(y_pred_test,axis=1))/len(y_test)
        print(i,L(T,y_pred,W1,W2),acc)